In [1]:
%run -i set_environment
%run -i importing_libraries
%run -i classifiers
%run -i importing_datasets

In [2]:
def fetch_datasets(dataset):
    
    data = arff.loadarff('./datasets/luis/' + dataset)
    metadata = data[1]
    data = pd.DataFrame(data[0])
    
    instances = len(data)
    classes = len(data.iloc[:,-1].value_counts())
    attributes = len(data.columns)- 1
    nominal_attributes = str(metadata).count("nominal")
    
    proportion = data.iloc[:,-1].value_counts()
    proportion = proportion.map(lambda x: round(x/instances*100,2))

    majority = max(proportion)
    minority = min(proportion)

    
    return {
        "name": dataset[:-5],
        "instances": instances,
        "classes": classes,
        "attributes": attributes,
        "nominal attributes": nominal_attributes,
        "majority": majority,
        "minority": minority
    }

In [10]:
def which_arff_dataset(dataset, n_splits = 5):
   
    from sklearn.preprocessing import OneHotEncoder
    
    data = arff.loadarff('datasets/luis/' + dataset)
    metadata = data[1]
    data = pd.DataFrame(data[0])

    X_raw = data[data.columns[:-1]]
    y_raw = data[data.columns[-1]].to_numpy()
    
    metadataSplit = str(metadata).split('\n')
    
    numericOrCategorical = []
    for x in range(1,len(data.columns)):
        if (metadataSplit[x].find('numeric')) == -1:
            numericOrCategorical.append(0) # categorical
        else:
            numericOrCategorical.append(1) # numeric
                               
    print(numericOrCategorical)

    print(X_raw)
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    enc.fit(X_raw)
    print(X_raw)
    print(enc.transform(X_raw))
    X_raw2 = pd.DataFrame()
    for x in range(len(data.columns)-1):
        print(x)
        if numericOrCategorical[x] == 0:
            X_raw2.append(enc.transform(X_raw.iloc(x)))
        else:
            X_raw2[X_raw.columns[x]] = X_raw.iloc(x)

#     lex = preprocessing.OrdinalEncoder()
#     lex.fit(X_raw)
#     X_raw = lex.transform(X_raw)

    print(X_raw2)
        
    ley = preprocessing.LabelEncoder()
    ley.fit(y_raw)
    y_raw = ley.transform(y_raw)
    
    # cross validation bags
    data_cv = StratifiedShuffleSplit(n_splits= n_splits, train_size=0.2, random_state=0) #n_splits
    data_cv.get_n_splits(X_raw,y_raw)
    
    # extraindo ids do data_cv
    idx_data = []
    for train_index, test_index in data_cv.split(X_raw, y_raw):
            idx_data.append([train_index, test_index])

    return X_raw, y_raw, idx_data, dataset

In [4]:
datasets = os.listdir('./datasets/luis')
classifiers = ['5NN', 'C4.5', 'NB','RF']

In [8]:
metadata = []

for ds in datasets:
    metadata.append(fetch_datasets(ds))

metadata = pd.DataFrame.from_dict(metadata)
metadata

,name,instances,classes,attributes,nominal attributes,majority,minority
0,1115_teachingAssistant,151,3,6,5,34.44,32.45


In [11]:
pyhard_strategies_names = ['H','U','H+U','LSC','N2','F3']

for ds in datasets:
    X_raw, y_raw, idx_data, dataset_name = which_arff_dataset(ds)

[1, 0, 0, 0, 0, 1]
        ID EnglishSepaker courseInstructor course summer  classSize
0      1.0           b'1'            b'23'   b'3'   b'1'       19.0
1      2.0           b'2'            b'15'   b'3'   b'1'       17.0
2      3.0           b'1'            b'23'   b'3'   b'2'       49.0
3      4.0           b'1'             b'5'   b'2'   b'2'       33.0
4      5.0           b'2'             b'7'  b'11'   b'2'       55.0
..     ...            ...              ...    ...    ...        ...
146  147.0           b'2'             b'3'   b'2'   b'2'       26.0
147  148.0           b'2'            b'10'   b'3'   b'2'       12.0
148  149.0           b'1'            b'18'   b'7'   b'2'       48.0
149  150.0           b'2'            b'22'   b'1'   b'2'       51.0
150  151.0           b'2'             b'2'  b'10'   b'2'       27.0

[151 rows x 6 columns]
        ID EnglishSepaker courseInstructor course summer  classSize
0      1.0           b'1'            b'23'   b'3'   b'1'       19.0
1    

ValueError: Expected 2D array, got scalar array instead:
array=<pandas.core.indexing._iLocIndexer object at 0x7fdb6b64c230>.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [82]:
def test1(a):
    a = 12
    return a
    
def test2():
    test1()
    a = 1243

In [75]:
def test_list_param(la):
    test2('b')
    la.append('a')

In [76]:
teste = {"test1": "test2"}

In [84]:
a = 1
a = eval(list(teste.keys())[0] + "(a)")
a

12

In [87]:
modal_strategies = {"Uncertain Sampling":"uncertain_sampling", "Random Sampling":"random_sampling", "Query by Committee":"query_by_committee", "Expected Error Reduction":"exp_error_reduction", "Expected Model Change":"exp_model_change"}
for k,v in modal_strategies.items():
    print(k,v)

Uncertain Sampling uncertain_sampling
Random Sampling random_sampling
Query by Committee query_by_committee
Expected Error Reduction exp_error_reduction
Expected Model Change exp_model_change
